## Übungsblatt 2

##### Einleitung + Packages
Okay Leute, wir müssen zwei Sachen zunächst implementieren, zunächst eine Operation, die nur Schüler ausgibt, bei denen die Noten über einem bestimmten Pegel lagen, die andere ist quasi ein Select-Befehl, aber als Code anstatt einer Anweisung.

Kopieren wir zunächst Sachen aus Blatt 1, die sich nicht ändern werden, wie etwa das Lesen oder Schreiben aus / in die Datei. Die Packages bleiben die gleichen, also schaut für Erklärung beim Package '**blatt01**' vorbei

In [1]:
import os
import csv
import numpy as np
import re

BASE_DIR: str = os.getcwd()
CSV_DIR: str = os.path.join(BASE_DIR, 'csv')


def read_csv_data(file_name: str) -> list:
    csv_data: list = []

    if not os.path.exists(CSV_DIR):
        os.makedirs(CSV_DIR)

    if file_name.endswith('.csv') and os.path.exists(file_name):
        with open(file=file_name, mode='r') as csv_file:
            csv_in = csv.reader(csv_file, quotechar='"', delimiter=";")
            for row in csv_in:
                csv_data.append(row)
    else:
        raise Exception("Path given doesn't point to a .csv file")

    return csv_data


def write_csv_data(csv_data: list, file_name: str) -> None:
    if os.path.exists(file_name):
        os.remove(file_name)

    with open(file=file_name, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, quotechar='"', delimiter=";")
        for row in csv_data:
            csv_writer.writerow(row)

#### ANALYZE
Fangen wir zuerst mit der Implementierung des Analyze-Befehls. Wir brauchen eine zusätzliche Operation, die eine Liste als Parameter, sowie einen Pegel bekommt und daraus bestimmt, ob eine Zeile gültig ist und in die finale Datei übernommen werden soll oder nicht.

- Row: List
    - Eine einzelne Zeile aus der CSV-Datei.
- G_Columns: List
    - Enthält die Indexe der Notenspalten.
- Threshold: Integer
    - Wird verwendet, um die Noten rauszufiltern, die unter dem Pegel liegen.

Wir holen uns die einzelnen Indexe der Noten aus G_Column und überprüfen, ob die Zeile an diesen Stellen gültige Werte enthält. Die Werte aus der Zeile müssen mit einem Typecast versehen werden, da sie sonst Strings sind. Standardmäßig ist Threshold = **0**.

In [2]:
def analyze_threshold(row: list, g_columns: list, threshold: int = 0) -> bool:

    for column_index in g_columns:

        if int(row[column_index]) < threshold:
            return False

    return True

Die Operation ist somit fertig, machen wir zunächst mit der Projektion weiter.

#### PROJECTION

Mit der Projection wollen wir uns bestimmte Zeilen ausgeben lassen, d.h. zum Beispiel nur die Noten der Schüler oder die Schulen auf die sie gehen.

- Row: List
    - Eine einzelne Zeile aus der CSV-Datei.
- S_Columns: List
    - Enthält String mit den Namen der Spalten, die rausgefiltert werden sollen.
- Column_Names: Dictionary
    - Hält die Spaltennamen, sowie ihre Indexe fest in der Form: {Spaltenname: Index}

Der Output muss in einer neuen Variable stehen. Wir überprüfen jeden Key aus dem Dictionary '**column_names**' auf Übereinstimmung mit den Strings, die in '**s_columns**' stehen. Sollte dies der Fall sein speichern wir den Wert von '**row**' mithilfe des Index aus dem Dictionary. Die Variable '**row_out**' hält die neue Liste und wird zurückgegeben, falls sie tatsächlich Werte enthält, sonst wird eine Exception geworfen.

In [3]:
def analyze_select(row: list, s_columns: list, column_names: dict) -> list:
    row_out: list = []
    for column_key in column_names:
        if column_key in s_columns:
            row_out.append(row[column_names[column_key]])

    if row_out:
        return row_out
    else:
        raise DTBException("Die Spaltensuche ergab keine Treffer, korrigieren Sie Ihre Angaben!")

#### Implementieren

Nun implementieren wir die Operationen in unsere Analyse-Operation.

Zuerst machen wir eine eigene Klasse, die von Exception erbt. Sie soll es erleichtern Fehler aufzufangen.

In [4]:
class DTBException(Exception):
    pass

Nun müssen in den Parametern der neuen Operation die nötigen Einträge hinzufügen. Diese wurden bereits erklärt, deswegen hier ein Copy-Paste:

- CSV-Data: List
    - Besteht aus weiteren Listen (Zeilen), die einzelne Werte (Zellen) halten.
- Threshold: Integer
    - Wird verwendet, um die Noten rauszufiltern, die unter dem Pegel liegen.
- S_Columns: List
    - Enthält String mit den Namen der Spalten, die rausgefiltert werden sollen.

Als Nächstes führen wir die Variable '**column_names**' ein. Die Unterschiede zu '**g_columns**' sind die Speicherung aller Spaltennamen, sowie die Art auf die die Werte gespeichert werden. Trotz der Tatsache, dass '**g_columns**' eine Teilmenge von '**column_names**' ist behalten wir beide, da wir sonst unnötige Rechenoperationen durchführen müssen, die die Laufzeit erhöhen.

Beim ersten Durchlauf der For-Schleife werden die Spaltennamen in '**column_names**' gespeichert und die Indexe der Notenspalten werden in '**g_columns**' gespeichert. Der RegEx hat sich nicht geändert.

Bei den sonstigen Durchläufen werden zunächst die Noten überprüft, sollten diese nicht stimmen müssen wir uns keine Mühe mit den Medians und Maximalwerten machen, wir können die Zeile einfach überspringen.

Besteht eine Zeile die Threshold-Prüfung so kann man die ganzen Durchschnitte, etc. berechnen. Nachdem die neue Zeile erstellt ist schneiden wir uns die nötigen Spalten raus, indem wir die '**analyze_select**' Operation aufrufen.

Zum Ende wird die fertige CSV ausgegeben.

In [5]:
def analyze_student_results(csv_data: list, s_columns: list = None, threshold: int = 0) -> list:
    csv_processed: list = []
    g_columns: list = []

    column_names: dict = {}

    for idx, row in enumerate(csv_data):

        if idx == 0:
            for g_idx, cell in enumerate(row):
                column_names[cell] = g_idx

                if re.match(r"^G[0-9]$", cell):
                    g_columns.append(g_idx)
            next_row = row + ['G_MAX', 'G_MEAN']

        else:
            if analyze_threshold(row=row, g_columns=g_columns, threshold=threshold):

                grades: list = []
                for g_column in g_columns:
                    grades.append(int(row[g_column]))
                g_max: int = np.max(grades)
                g_mean: float = round(np.average(grades), 2)

                next_row: list = row + [str(g_max), str(g_mean)]
            else:
                continue

        csv_processed.append(analyze_select(row=next_row, column_names=column_names, s_columns=s_columns))

    return csv_processed

#### Testlauf

Testen wir den Scheiß. Wir laden die Datei '**student-mat.csv**' und erstellen eine kleine Select-Anweisung, die uns die '**famsize**', was auch immer das sein soll, sowie das Geschlecht ausgibt.

Wir lesen die Datei, verarbeiten sie und geben sie aus. Es funktioniert alles.

![Nice](https://media.tenor.com/NQ0nRbi_REIAAAAM/awesome-computer-kid-computer.gif)

In [6]:
FILE_IN = os.path.join(CSV_DIR, 'student-mat.csv')
STUDENT_AMOUNT = sum(1 for line in open(FILE_IN)) - 1
FILE_OUT = os.path.join(CSV_DIR, f'student-mat-out-total-{STUDENT_AMOUNT}.csv')

S_COLUMNS = ['famsize', 'sex']

csv_data = read_csv_data(file_name=FILE_IN)
csv_data_processed = analyze_student_results(csv_data=csv_data, s_columns=S_COLUMNS, threshold=7)
write_csv_data(csv_data=csv_data_processed, file_name=FILE_OUT)

Created by [Arnold Schwarzenegger](https://www.instagram.com/schwarzenegger/)